In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import datetime
from hatch import create_token_batches, TokenBatch, Commons
from convictionvoting import trigger_threshold
from policies import *
from network_utils import *
from IPython.core.debugger import set_trace
from entities import Participant, Proposal
from cadCAD.configuration import Configuration
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

In [2]:
# For the Flask backend

# Commons/Augmented Bonding Curve parameters
hatchers = 60
proposals = 3
hatch_tribute = 0.2
vesting_80p_unlocked = 60
exit_tribute = 0.35
# kappa = 2, default option set in abcurve.py, there is no way to reach it from here for now

# Conviction Voting parameters
days_to_80p_of_max_voting_weight = 10  # used in ProposalFunding.su_calculate_gathered_conviction
max_proposal_request = 0.2  # will be passed to trigger_threshold()

In [3]:
def update_collateral_pool(params, step, sL, s, _input):
    commons = s["commons"]
    s["collateral_pool"] = commons._collateral_pool
    return "collateral_pool", commons._collateral_pool


def update_token_supply(params, step, sL, s, _input):
    commons = s["commons"]
    s["token_supply"] = commons._token_supply
    return "token_supply", commons._token_supply


def update_funding_pool(params, step, sL, s, _input):
    commons = s["commons"]
    s["funding_pool"] = commons._funding_pool
    return "funding_pool", commons._funding_pool

In [4]:
contributions = [np.random.rand() * 10e5 for i in range(hatchers)]
token_batches, initial_token_supply = create_token_batches(contributions, 0.1, vesting_80p_unlocked)

commons = Commons(sum(contributions), initial_token_supply, hatch_tribute=0.2, exit_tribute=0.35)
network = bootstrap_network(token_batches, proposals, commons._funding_pool, commons._token_supply, max_proposal_request)

initial_conditions = {
    "network": network,
    "commons": commons,
    "funding_pool": commons._funding_pool,
    "collateral_pool": commons._collateral_pool,
    "token_supply": commons._token_supply,
    "sentiment": 0.5,
}

partial_state_update_blocks = [
    {
        "policies": {
            "generate_new_participants": GenerateNewParticipant.p_randomly,
        },
        'variables': {
            'network': GenerateNewParticipant.su_add_to_network,
            'commons': GenerateNewParticipant.su_add_investment_to_commons,
        }
    },
    {
        "policies": {},
        "variables": {
            "funding_pool": update_funding_pool,
            "collateral_pool": update_collateral_pool,
            "token_supply": update_token_supply,
        }
    },
    {
        "policies": {
            "generate_new_proposals": GenerateNewProposal.p_randomly,
        },
        "variables": {
            "network": GenerateNewProposal.su_add_to_network,
        }
    },
    {
        "policies": {
            "generate_new_funding": GenerateNewFunding.p_exit_tribute_of_average_speculator_position_size,
        },
        "variables": {
            "network": GenerateNewFunding.su_add_funding,
        }
    },
    {
        "policies": {},
        "variables": {
            "network": ProposalFunding.su_update_age_and_conviction_thresholds,
        }
    },
    {
        "policies": {},
        "variables": {
            "network": ProposalFunding.su_update_gathered_conviction,
        }
    },
    {
        "policies": {
            "decide_which_proposals_should_be_funded": ProposalFunding.p_compare_conviction_and_threshold
        },
        "variables": {
            "network": ProposalFunding.su_compare_conviction_and_threshold_make_proposal_active,
            "commons": ProposalFunding.su_compare_conviction_and_threshold_deduct_funds_from_funding_pool,
        }
    },
]

In [ ]:

# TODO: make it explicit that 1 timestep is 1 day
simulation_parameters = {
    'T': range(150),
    'N': 1,
    'M': {
        # "sentiment_decay": 0.01, #termed mu in the state update function
        # "trigger_threshold": trigger_threshold,
        # "min_proposal_age_days": 7, # minimum periods passed before a proposal can pass,
        # "sentiment_sensitivity": 0.75,
        # 'min_supp':50, #number of tokens that must be stake for a proposal to be a candidate
        "debug": True,
        "days_to_80p_of_max_voting_weight": days_to_80p_of_max_voting_weight,
        "max_proposal_request": max_proposal_request,
    }
}

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state=initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_config=simulation_parameters #dict containing simulation parameters
                      )

exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc) # Do not use multi_proc, breaks ipdb.set_trace()
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results


                            __________   ____ 
          ________ __ _____/ ____/   |  / __ \
         / ___/ __` / __  / /   / /| | / / / /
        / /__/ /_/ / /_/ / /___/ ___ |/ /_/ / 
        \___/\__,_/\__,_/\____/_/  |_/_____/  
        by BlockScience
        
Execution Mode: single_proc: [<cadCAD.configuration.Configuration object at 0x7f12649b7700>]
Configurations: [<cadCAD.configuration.Configuration object at 0x7f12649b7700>]
ProposalFunding: Proposal 60 has 1691549.7495683844 conviction, and needs 162678520.5991798 to pass
ProposalFunding: Proposal 61 has 1907893.6208319818 conviction, and needs 156479343.4193817 to pass
ProposalFunding: Proposal 62 has 1978837.1931115014 conviction, and needs 159369692.55634004 to pass
ProposalFunding: Proposal 60 has 18268081.465242684 conviction, and needs 162677547.09710097 to pass
ProposalFunding: Proposal 61 has 20604511.396291304 conviction, and needs 156479007.12307543 to pass
ProposalFunding: Proposal 62 has 21370674.471405283 con

ProposalFunding: Proposal 84 has 276578610.3802805 conviction, and needs 173765143.11701605 to pass
ProposalFunding: Proposal 84 requested 43668.44266431387 funds, deducting from Commons funding pool
ProposalFunding: Proposal 85 has 223404.7986978979 conviction, and needs 191546389.99467254 to pass
ProposalFunding: Proposal 85 has 2457452.785676877 conviction, and needs 191537829.10981542 to pass
ProposalFunding: Proposal 85 has 24786285.84532224 conviction, and needs 191530695.19327256 to pass
ProposalFunding: Proposal 86 has 262118.3159397444 conviction, and needs 263519625.3724291 to pass
ProposalFunding: Proposal 85 has 248074616.44177592 conviction, and needs 191522351.57194614 to pass
ProposalFunding: Proposal 86 has 2883301.4753371887 conviction, and needs 263489423.80529228 to pass
ProposalFunding: Proposal 85 requested 74463.90645752818 funds, deducting from Commons funding pool
ProposalFunding: Proposal 86 has 29095133.069311623 conviction, and needs 267056616.88954052 to pas

ProposalFunding: Proposal 104 has 132786435.86184892 conviction, and needs 185366480.74601242 to pass
ProposalFunding: Proposal 104 has 1327982016.00269 conviction, and needs 185357073.7447054 to pass
ProposalFunding: Proposal 104 requested 45360.17566181684 funds, deducting from Commons funding pool
ProposalFunding: Proposal 107 has 72851.8556594398 conviction, and needs 385016261.2835071 to pass
ProposalFunding: Proposal 107 has 798908.3659472199 conviction, and needs 384888196.1554623 to pass
ProposalFunding: Proposal 108 has 135923.4180591489 conviction, and needs 164762708.66780162 to pass
ProposalFunding: Proposal 107 has 8059473.468825021 conviction, and needs 384780665.29337454 to pass
ProposalFunding: Proposal 108 has 1495157.5986506382 conviction, and needs 164759759.95867562 to pass
ProposalFunding: Proposal 107 has 80665124.49760301 conviction, and needs 384652804.1657306 to pass
ProposalFunding: Proposal 108 has 15087499.40456553 conviction, and needs 164756252.2473971 to 

In [ ]:
df = pd.DataFrame(raw_result)
df_final = df[df.substep.eq(2)]


In [ ]:
df_final.plot("timestep", "collateral_pool", grid=True)
df_final.plot("timestep", "token_supply", grid=True)
df_final.plot("timestep", "funding_pool", grid=True)

In [ ]:
# import matplotlib.pyplot as plt
# supporters = get_edges_by_type(network, 'support')
# influencers = get_edges_by_type(network, 'influence')
# competitors = get_edges_by_type(network, 'conflict')

# nx.draw_kamada_kawai(network, nodelist = get_participants(network), edgelist=influencers)
# plt.title('Participants Social Network')

In [ ]:
# For the Flask backend
result = {
    "timestep": list(df_final["timestep"]),
    "funding_pool": list(df_final["funding_pool"]),
    "token_supply": list(df_final["token_supply"]),
    "collateral": list(df_final["collateral_pool"])
}